In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.models import alexnet, AlexNet_Weights
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import utils

torch.manual_seed(42069)
# if gpu available else cpu
device = torch.device(0) if torch.cuda.is_available() else torch.device('cpu')
TRAIN_MEAN = [0.5036, 0.4719, 0.3897]
TRAIN_STD = [0.2623, 0.2577, 0.2671]
classes = ['butterfly','cat', 'chicken', 'cow', 'dog', 'elephant', 'horse', 'sheep', 'spider', 'squirrel']

## Przygotowanie danych

In [2]:
# zdefiniowanie operacji na kazdym obrazie w zbiorze
transform = transforms.Compose([
    transforms.RandomResizedCrop(256),  #  na 256x256
    transforms.RandomHorizontalFlip(),  # wycinamy losowy fragment 128x128
    transforms.ToTensor(),              # obrazy zamieniamy na tensory,
    # srednie i odchylenia po kanałach całęgo zbioru,
    #  wyliczone wczesniej za pomocą utils.data_normalize_values
    transforms.Normalize(TRAIN_MEAN, TRAIN_STD)
])

# loader danych z batchami
train_data = ImageFolder(root='dataset/train/', transform=transform)
test_data = ImageFolder(root='dataset/test/', transform=transform)
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size)

## Uczenie klasyfikatora

In [3]:
# RESNET18 z wagami przetrenowanymi na zbiorze IMAGENET
alex_net_model = alexnet(weights=AlexNet_Weights.IMAGENET1K_V1)
# dostosowanie ostatniego layeru do problemu
alex_net_model.classifier[6] = nn.Linear(alex_net_model.classifier[6].in_features, len(classes))
# inicjalizacja wag w warstwie wyjsciowej
nn.init.xavier_uniform_(alex_net_model.classifier[6].weight)


alex_net_model = alex_net_model.to(device)
metrics = utils.train_fine_tuning(
    model = alex_net_model, 
    learning_rate= 5e-5, 
    train_loader=train_loader,
    test_loader=test_loader,
    device=device, num_epochs=5, param_group=True
)

Progress: 1/5 epochs
Epoch: 1, Loss 0.638, Train acc: 0.788, Test acc: 0.841
Progress: 2/5 epochs
Epoch: 2, Loss 0.48, Train acc: 0.839, Test acc: 0.851
Progress: 3/5 epochs
Epoch: 3, Loss 0.455, Train acc: 0.845, Test acc: 0.862
Progress: 4/5 epochs
Epoch: 4, Loss 0.43, Train acc: 0.855, Test acc: 0.869
Progress: 5/5 epochs
Epoch: 5, Loss 0.409, Train acc: 0.858, Test acc: 0.873
